In [1]:
!pip install openai openai-agents pydantic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.9/193.9 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 948.4/948.4 kB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.4/144.4 kB 4.8 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.107.0
    Uninstalling openai-1.107.0:
      Successfully uninstalled openai-1.107.0


In [12]:
!pip install sendgrid

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 63.0 MB/s eta 0:00:00
  Attempting uninstall: cryptography
    Found existing installation: cryptography 43.0.3
    Uninstalling cryptography-43.0.3:
      Successfully uninstalled cryptography-43.0.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pyopenssl 24.2.1 requires cryptography<44,>=41.0.5, but you have cryptography 46.0.1 which is incompatible.
pydrive2 1.21.3 requires cryptography<44, but you have cryptography 46.0.1 which is incompatible.


In [104]:
from agents import Agent, Runner, trace, OpenAIChatCompletionsModel, GuardrailFunctionOutput, function_tool, input_guardrail
from openai import AsyncOpenAI
from pydantic import BaseModel
import os
from google.colab import userdata
import sendgrid
from sendgrid import SendGridAPIClient
from sendgrid.helpers.mail import To, Email, Mail, Content


In [79]:
gemini_base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
groq_base_url="https://api.groq.com/openai/v1"

gemini_api_key=userdata.get("GOOGLE_API_KEY")
groq_api_key=userdata.get("GROQ_API_KEY")

In [80]:
gemini_client=AsyncOpenAI(api_key=gemini_api_key, base_url=gemini_base_url)
groq_client=AsyncOpenAI(api_key=groq_api_key, base_url=groq_base_url)

gemini_model=OpenAIChatCompletionsModel(model="gemini-2.5-flash", openai_client=gemini_client)
groq_model=OpenAIChatCompletionsModel(model="openai/gpt-oss-20b", openai_client=groq_client)

# Instructions for Agents

In [81]:
inst1="You are a sales agent working for ComplAI, \
a company that provides SaaS tool for ensuring SOC2 compilance and preparing audits, powered by AI. \
You write engaging cold sales emails that are likely to get reponse from user."

inst2="You are a sales agent working for ComplAI, \
a company that provides SaaS tool for ensuring SOC2 compilance and preparing audits, powered by AI. \
You write professional and serious cold sales email"

Sales_Agent1=Agent(
    name="Sales_Agent1",
    instructions=inst1,
    model=gemini_model
)

Sales_Agent2=Agent(
    name="Sales_Agent2",
    instructions=inst2,
    model=gemini_model
)

In [82]:
subject_inst="You are a subject extracter from given email body, \
you have to extract best subject of email based on given email body"

subject_agent=Agent(
    name="subject_agent",
    instructions=subject_inst,
    model=gemini_model
)

In [83]:
html_inst="you are a email converter agent from text email body to HTML email body. \
you have to precisely convert text email to html email, text may include markdown."

html_agent=Agent(
    name="html_agent",
    instructions=html_inst,
    model=gemini_model
)

# UserError: Hosted tools are not supported with the ChatCompletions API.
# Got tool type: <class 'agents.agent.Agent'>, tool: Agent(name='html_agent',

# Agent as tools

In [92]:
tool1=Sales_Agent1.as_tool(tool_name="Sales_Agent1",tool_description="Write engaging cold sales email for company ComplAI")
tool2=Sales_Agent2.as_tool(tool_name="Sales_Agent2",tool_description="Write professional and serious cold sales email for company ComplAI")
subject_agent_tool=subject_agent.as_tool(tool_name="subject_agent",tool_description="give best subject for given email")
#html_agent_tool=html_agent.as_tool(tool_name="html_agent",tool_description="Convert text email body to HTML email body")

## Functions as tool

In [93]:
@function_tool
def send_email(subject, body:str):
  """Send out email with the given HTML email body to all the sales prospects"""
  sg=SendGridAPIClient(api_key=userdata.get("SENDGRID_API_KEY"))

  from_email=Email("muasif025@gmail.com")
  to_email=To("as29624041@gmail.com")
  content=Content("text/plain",body)
  mail=Mail(from_email, to_email, subject ,content).get()
  response=sg.client.mail.send.post(request_body=mail)

  return {"status":"success"}

In [94]:
tools=[tool1, tool2, subject_agent_tool, send_email]

In [88]:
tools

[FunctionTool(name='Sales_Agent1', description='Write engaging cold sales email for company ComplAI', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'Sales_Agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x7edf1e470cc0>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='Sales_Agent2', description='Write professional and serious cold sales email for company ComplAI', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'Sales_Agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x7edf1f2744a0>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='subject_agent', description='give best subject for given email', params_

In [95]:
manager_inst="You are a Sales manager working for ComplAI. \
You use the tools given to you to generate cold sales email. \
You never generate sales emails yourself; you always use the tools. \
You try all 2 sales_agent tools once before choosing the best one. \
You pick the single best email and use the subject_agent tool to extract subject of email,\
and use the send_email tool to send the best email (and only the best email) to the user with the agent given subject"

manager_agent=Agent(
    name="Sales manager",
    instructions=manager_inst,
    tools=tools,
    model=gemini_model
)


In [96]:
message="Send out a cold sales email addressed to Dear CEO from Alice"
result=await Runner.run(manager_agent,message)

In [97]:
result.final_output

'The email has been sent.'

# Agent Guardrails

In [99]:
class NameCheckOutput(BaseModel):
  is_name_in_message:bool
  name:str

In [103]:
guardrail_agent=Agent(
    name="Name Check",
    instructions="check if the user is including someone's personal name in what they want you to do.",
    output_type=NameCheckOutput,
    model=gemini_model
)

In [106]:
@input_guardrail
async def guardrail_against_name(ctx, agent, message):
  result= await Runner.run(guardrail_agent, message, context=ctx.context)
  is_name_in_message=result.final_output.is_name_in_message

  return GuardrailFunctionOutput(output_info={"found_name": result.final_output}, tripwire_triggered=is_name_in_message)

In [108]:
manager_inst="You are a Sales manager working for ComplAI. \
You use the tools given to you to generate cold sales email. \
You never generate sales emails yourself; you always use the tools. \
You try all 2 sales_agent tools once before choosing the best one. \
You pick the single best email and use the subject_agent tool to extract subject of email,\
and use the send_email tool to send the best email (and only the best email) to the user with the agent given subject"

careful_manager_agent=Agent(
    name="Sales manager",
    instructions=manager_inst,
    tools=tools,
    model=gemini_model,
    input_guardrails=[guardrail_against_name]
)


In [110]:
message="Send out a cold sales email"
result=await Runner.run(careful_manager_agent,message)

In [111]:
print(result.final_output)

The cold sales email has been sent with the subject "Free Up Developer Time from SOC2 Compliance (with ComplAI)".


In [112]:
message="Send out a cold sales email addressed to Dear CEO from Alice"
result=await Runner.run(careful_manager_agent,message)

InputGuardrailTripwireTriggered: Guardrail InputGuardrail triggered tripwire

In [113]:
message="Send out a cold sales email addressed to Dear CEO Head of Business Department"
result=await Runner.run(careful_manager_agent,message)

In [114]:
print(result.final_output)

I have sent out a cold sales email with the subject "Enhancing SOC2 Compliance & Audit Readiness: A Strategic Advantage for [Company Name]" to the CEO Head of Business Department.
